In [7]:
from itertools import permutations, product

import tqdm

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
from os import path

import matplotlib.pyplot as plt

from warnings import simplefilter
from collections import OrderedDict

if (os.path.abspath('').split('/')[-1] == 'project'):
    %cd utils
elif (os.path.abspath('').split('/')[-1] == 'train_and_vis'):
    %cd ../utils

import query_utils
import model_utils
import validation_utils

print(os.path.abspath(''))

if (os.path.abspath('').split('/')[-1] == 'utils'):
    %cd ..


simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [8]:
nard_seq = pd.read_csv("data/nardus_sequences.csv")
# nard_seq.drop(0, axis=1, inplace=True)

In [9]:
print(nard_seq)

               ID                                       DNA Sequence  \
0     NC_025403.1  accagagggaaaatataacaatgtcgttttatagcgatgtaaataa...   
1     NC_025404.1  accagagggaaaattaagaaaggtcgttccaagacgacttaaaaga...   
2     NC_028246.1  acggagaaaaacaaaaaaactatagtgattagataaataaggaaaa...   
3     NC_002077.1  ttgcccactccctctctgcgcgctcgctcgctcggtggggcctgcg...   
4     NC_006152.1  ctctcccccctgtcgcgttcgctcgctcgctggctcgtttgggggg...   
...           ...                                                ...   
1843  NC_007661.1  gttaaaactctcacacttatggtggaactggatctgcaaaaatggg...   
1844  NC_007662.1  gttaaaaatctggttgtatctactcttgaatgaacgtgcataaagc...   
1845  NC_007663.1  gttaaaatctggaaccaatatggaagggatttatgcacgtgctttt...   
1846  NC_007664.1  gttaaaaaagagtgcagatgtcgagaatagtcttactaacaccagg...   
1847  NC_007665.1  gttaaaaacctcaagatgcatgccgctattacttcaatgaaacgtg...   

      isZoonotic  
0              0  
1              0  
2              0  
3              1  
4              1  
...          ...  
18

In [10]:
df = pd.read_csv('data/virus_genome/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1

df2 = pd.read_csv('data/virus_genome/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0

dfother = pd.DataFrame()
it = len(df)

# iterate through each sequence
for i in range(len(df2)):
    dfother.at[it, 'DNA Sequence']=df2.at[i, 'DNA Sequence']
    dfother.at[it, 'isZoonotic']=int(0)
    dfother.at[it, 'ID']=df2.at[i, 'ID']
    it+=1


df.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

mergedDf = pd.concat([df, dfother], axis=0)

In [11]:
# 1848
print(df.columns)
print(nard_seq.columns)

Index(['ID', 'DNA Sequence', 'isZoonotic'], dtype='object')
Index(['ID', 'DNA Sequence', 'isZoonotic'], dtype='object')


In [12]:
print(mergedDf)

           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
2       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
3      120686  ttgttctacttcttactcattattataaattataatgtttgtataa...         1.0
4       99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...         1.0
...       ...                                                ...         ...
9423   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9424   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9425  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9426   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9427    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9428 rows x 3 columns]


In [18]:
x = 0
y=0
for col in nard_seq.iterrows():
    print(col[1])
    # if seq in mergedDf['DNA Sequence'].values:
    #     x+=len(mergedDf[mergedDf['DNA Sequence'] == seq])
    #     mergedDf = mergedDf[mergedDf['DNA Sequence'] != seq]
    # else:
    #     # append to the existing DF
    #     mergedDf.loc[len(mergedDf.index)]=[None, seq, ]
    #     y+=1
print("removed", x, "need to add: ", y)

ID                                                    NC_025403.1
DNA Sequence    accagagggaaaatataacaatgtcgttttatagcgatgtaaataa...
isZoonotic                                                      0
Name: 0, dtype: object
ID                                                    NC_025404.1
DNA Sequence    accagagggaaaattaagaaaggtcgttccaagacgacttaaaaga...
isZoonotic                                                      0
Name: 1, dtype: object
ID                                                    NC_028246.1
DNA Sequence    acggagaaaaacaaaaaaactatagtgattagataaataaggaaaa...
isZoonotic                                                      0
Name: 2, dtype: object
ID                                                    NC_002077.1
DNA Sequence    ttgcccactccctctctgcgcgctcgctcgctcggtggggcctgcg...
isZoonotic                                                      1
Name: 3, dtype: object
ID                                                    NC_006152.1
DNA Sequence    ctctcccccctgtcgcgttcgctcgctcgctggc

In [5]:
print(len(mergedDf))

8860
